In [ ]:
import cv2
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import os

OUTPUT_FOLDER = "tracking_results"
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# PART 1: MATH & PLOTTING 

def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def from_scratch_derivatives(position_track, fps):
    if len(position_track) < 5:
        return (np.array([]), np.array([]), np.array([]), np.array([]))
        
    dt = 1.0 / fps
    positions = np.array(position_track)
    
    velocity = np.diff(positions, axis=0) / dt
    speed = np.linalg.norm(velocity, axis=1)
    
    acceleration_vec = np.diff(velocity, axis=0) / dt
    acceleration = np.linalg.norm(acceleration_vec, axis=1)
    
    jerk_vec = np.diff(acceleration_vec, axis=0) / dt
    jerk = np.linalg.norm(jerk_vec, axis=1)
    
    jounce_vec = np.diff(jerk_vec, axis=0) / dt
    jounce = np.linalg.norm(jounce_vec, axis=1)
    
    return speed, acceleration, jerk, jounce

def save_derivative_plot(track_id, speed, acceleration, jerk, jounce, fps):
    if speed.size == 0: return
    
    time_speed = np.arange(len(speed)) / fps
    time_acc = np.arange(len(acceleration)) / fps
    time_jerk = np.arange(len(jerk)) / fps
    time_jounce = np.arange(len(jounce)) / fps
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(f'Motion Derivatives for Track ID: {track_id}', fontsize=16)
    
    ax1.plot(time_speed, speed, 'b-', label='Speed')
    ax1.set_title('Speed')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(time_acc, acceleration, 'r-', label='Acceleration')
    ax2.set_title('Acceleration')
    ax2.grid(True, alpha=0.3)
    
    ax3.plot(time_jerk, jerk, 'g-', label='Jerk')
    ax3.set_title('Jerk')
    ax3.grid(True, alpha=0.3)
    
    ax4.plot(time_jounce, jounce, 'm-', label='Jounce')
    ax4.set_title('Jounce')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    filename = f"{OUTPUT_FOLDER}/track_{track_id}.png"
    plt.savefig(filename)
    plt.close(fig)
    print(f"Saved graph for Track {track_id} to {filename}")

# PART 2: ROBUST TRACKER WITH "PATIENCE"

class RobustTracker:
    def __init__(self, blur_ksize, threshold_val, min_contour_area, max_track_distance, patience):
        self.blur_ksize = blur_ksize
        self.threshold_val = threshold_val
        self.min_contour_area = min_contour_area
        self.max_track_distance = max_track_distance
        self.patience_limit = patience # How many frames to wait before killing a track
        
        self.prev_frame_gray = None
        
        # self.tracks structure: {track_id: [list_of_positions]}
        self.tracks = {} 
        
        # self.track_patience structure: {track_id: current_patience_counter}
        self.track_patience = {} 
        
        self.next_track_id = 0
        self.frame_number = 0

    def _preprocess_frame(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, self.blur_ksize, 0)
        return blurred

    def _update_tracks(self, current_centroids):
        self.frame_number += 1
        
        # 1. If no existing tracks, initialize all found centroids
        if not self.tracks:
            for centroid in current_centroids:
                self.tracks[self.next_track_id] = [centroid]
                self.track_patience[self.next_track_id] = self.patience_limit
                self.next_track_id += 1
            return

        # 2. Prepare for matching
        active_track_ids = list(self.tracks.keys())
        last_positions = [self.tracks[tid][-1] for tid in active_track_ids]
        
        # Using a set to keep track of which NEW centroids have been assigned
        matched_new_centroids = set()
        matched_track_ids = set()

        # 3. Match existing tracks to new centroids (Greedy Nearest Neighbor)
        # We iterate through tracks and find their closest centroid
        for idx, track_id in enumerate(active_track_ids):
            last_pos = last_positions[idx]
            
            best_dist = float('inf')
            best_centroid_idx = -1
            
            for i, centroid in enumerate(current_centroids):
                if i in matched_new_centroids:
                    continue
                
                dist = euclidean_distance(last_pos, centroid)
                if dist < best_dist:
                    best_dist = dist
                    best_centroid_idx = i
            
            # Check if the match is valid
            if best_dist < self.max_track_distance and best_centroid_idx != -1:
                # --- UPDATE TRACK ---
                self.tracks[track_id].append(current_centroids[best_centroid_idx])
                self.track_patience[track_id] = self.patience_limit # Reset patience
                
                matched_new_centroids.add(best_centroid_idx)
                matched_track_ids.add(track_id)
        
        # 4. Handle LOST Tracks (The "Patience" Logic)
        for track_id in active_track_ids:
            if track_id not in matched_track_ids:
                # Ensure the patience key exists
                if track_id not in self.track_patience:
                     self.track_patience[track_id] = 0
                     
                # Decrease patience
                self.track_patience[track_id] -= 1
                
                # Visualize that we are "coasting" (optional, just repeats last pos)
                # self.tracks[track_id].append(self.tracks[track_id][-1]) 
                
                # If patience runs out, we don't delete the data, 
                # but we essentially stop updating it.
                # (We handle final cleanup at the end of the video or via separate logic)
                pass

        # 5. Handle NEW Objects
        for i, centroid in enumerate(current_centroids):
            if i not in matched_new_centroids:
                self.tracks[self.next_track_id] = [centroid]
                self.track_patience[self.next_track_id] = self.patience_limit
                self.next_track_id += 1

    def _draw_overlay(self, frame, num_objects, processing_time):
        cv2.putText(frame, f"Objects: {num_objects}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        for track_id, positions in self.tracks.items():
            # Only draw active tracks (patience > 0)
            if self.track_patience.get(track_id, 0) > 0:
                last_pos = positions[-1]
                cv2.circle(frame, last_pos, 6, (0, 0, 255), -1)
                cv2.putText(frame, f"ID:{track_id}", (last_pos[0]+10, last_pos[1]), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        return frame

    def process_video(self, video_path, output_video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Error opening video")
            return None, 0
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        
        ret, first_frame = cap.read()
        if not ret: return None, 0
        
        self.prev_frame_gray = self._preprocess_frame(first_frame)
        
        
        while cap.isOpened():
            start_time = time.time()
            ret, frame = cap.read()
            if not ret: break
            
            # 1. Preprocess
            current_frame_gray = self._preprocess_frame(frame)
            
            # 2. Motion Mask
            diff_frame = cv2.absdiff(current_frame_gray, self.prev_frame_gray)
            _, motion_mask = cv2.threshold(diff_frame, self.threshold_val, 255, cv2.THRESH_BINARY)
            
            # --- NEW STEP: DILATION (The "Snow Glue") ---
            # This connects the skier to his skis and ignores small gaps
            kernel = np.ones((5, 5), np.uint8) 
            motion_mask = cv2.dilate(motion_mask, kernel, iterations=2)
            
            # 3. Find Contours (Back to contours as they are faster/easier for simple blob tracking)
            contours, _ = cv2.findContours(motion_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            current_centroids = []
            
            for contour in contours:
                if cv2.contourArea(contour) < self.min_contour_area:
                    continue
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    current_centroids.append((cX, cY))
            
            # 4. Update Tracks
            self._update_tracks(current_centroids)
            
            proc_ms = (time.time() - start_time) * 1000
            
            # 5. Draw & Save
            display_frame = self._draw_overlay(frame, len(current_centroids), proc_ms)
            writer.write(display_frame)
            cv2.imshow("Robust Tracking", display_frame)
            
            self.prev_frame_gray = current_frame_gray
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        writer.release()
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
        return self.tracks, fps

    def analyze_tracks(self, fps):
        print("\n--- Post-Processing Analysis ---")
        print(f"Saving graphs to '{OUTPUT_FOLDER}'...")
        
        count = 0
        for track_id, positions in self.tracks.items():
            # Filter: Track must be decent length AND recently active
            # We check if patience > -10 to ensure we aren't analyzing old dead tracks
            if len(positions) > 30: 
                save_derivative_plot(track_id, positions, fps)
                count += 1
        
        if count == 0:
            print("No long tracks found. Try lowering threshold or increasing max_track_distance.")
            
    # Helper to simplify the call in analyze_tracks
    def analyze_tracks_wrapper(self, fps):
        print("\n--- Post-Processing Analysis ---")
        for track_id, positions in self.tracks.items():
            if len(positions) < 30: continue
            
            # Recalculate derivatives here to pass to plotter
            s, a, j, s4 = from_scratch_derivatives(positions, fps)
            save_derivative_plot(track_id, s, a, j, s4, fps)

if __name__ == "__main__":
    # --- TUNING FOR SKIER ---
    # MAX_TRACK_DISTANCE: Increased to 150 because skiers move fast in a flip
    # PATIENCE: Set to 10 frames. If he disappears for 0.3 seconds, we still keep his ID.
    
    tracker = RobustTracker(
        blur_ksize=(15, 15),
        threshold_val=25,
        min_contour_area=200,    
        max_track_distance=150, # Increased for fast movement
        patience=15             # Wait 15 frames before losing ID
    )
    
    tracks, video_fps = tracker.process_video('conveir_belt.mp4', 'processed_video.mp4')
    
    if tracks:
        tracker.analyze_tracks_wrapper(video_fps)